# IV.H - Tampering Telltales - Transient Attributes XAI

Code for estimating the transient attributes under different timestamps

In [ ]:
# !pip install --upgrade scikit-image
# !pip install pyproj

### Imports & Definitions

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os, sys

from itertools import product
import matplotlib.image as mpimg
import pyproj

from skimage import transform

sys.path.append("../../datasets")
from dataLoader import DataLoader, preprocess_time

from tf.keras.models import Model, load_model
from tf.keras.layers import Input
from tf.keras.losses import mean_squared_error

transLabels = ["dirty", "daylight", "night", "sunrisesunset", "dawndusk", "sunny", "clouds", 
               "fog", "storm", "snow", "warm", "cold", "busy", "beautiful", "flowers", "spring", 
               "summer", "autumn", "winter", "glowing", "colorful", "dull", "rugged", "midday", 
               "dark", "bright", "dry", "moist", "windy", "rain", "ice", "cluttered", "soothing", 
               "stressful", "exciting", "sentimental", "mysterious", "boring", "gloomy", "lush"]


batchSize = 1
pathToModel = "../IV.B_ablation_study/denseNet/gr_oh_loc_time_TA/weights.30-0.57407.hdf5"
gpuNumber = 4

## GPU selection
import tensorflow as tf 
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[gpuNumber], 'GPU')
tf.config.experimental.set_memory_growth(gpus[gpuNumber], False)



#######################
##    Custom MSE     ##
#######################
# We will compute the MSE only for the consistent inputs
def transient_mse(y_true, y_pred):
    return tf.sum(mean_squared_error(y_true[0::2,:], y_pred[0::2,:]), axis=-1)



#######################
## Deprocess time
#######################
def deprocess_time(time):
    month, hour = time
    month = (11.0 / 2.0) * (month + 1) + 1
    hour = (23.0 / 2.0) * (hour + 1)
    return (int(round(month)), int(round(hour)))

def deprocess_loc(loc):
    _earth_radius = 6378137.0
    x,y,z = loc
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    
    lon, lat, alt = pyproj.transform(ecef, lla, x * _earth_radius, y*_earth_radius, z*_earth_radius, radians=False)
    return (lat, lon, alt)

Using TensorFlow backend.


### Load architecture and get pointers to specific layers
As we will process the features from multiple timestamps, we avoid re-processing the features for the ground-level image, location, and satellite image

In [3]:
baseModel = load_model(pathToModel, custom_objects={"transient_mse": transient_mse})
print(baseModel.summary())

groundBranchModel = Model(baseModel.get_layer("groundInput").input, 
                          baseModel.get_layer("batch_normalization_2").output)
aerialBranchModel = Model(baseModel.get_layer("aerialInput").input, 
                          baseModel.get_layer("batch_normalization_4").output)
locBranchModel = Model(baseModel.get_layer("locationInput").input, 
                       baseModel.get_layer("batch_normalization_7").output)
timeBranchModel =  Model(baseModel.get_layer("timeInput").input, 
                         baseModel.get_layer("batch_normalization_10").output)

combinedFeaturesInput = Input(shape=(512,), name='concatenate_1_proxy')
consistFeatures = baseModel.get_layer("consist_fc1")(combinedFeaturesInput)
consistFeatures = baseModel.get_layer("batch_normalization_11")(consistFeatures)
consistFeatures = baseModel.get_layer("consist_fc2")(consistFeatures)
consistFeatures = baseModel.get_layer("batch_normalization_12")(consistFeatures)
consistFeatures = baseModel.get_layer("consist_fc3")(consistFeatures)
consistModel = Model(combinedFeaturesInput, consistFeatures)


grFeaturesInput = Input(shape=(128,), name='grFeaturesProxy')
grTransFeatures = baseModel.get_layer("gr_trans_fc1")(grFeaturesInput)
grTransFeatures = baseModel.get_layer("batch_normalization_13")(grTransFeatures)
grTransFeatures = baseModel.get_layer("gr_trans_fc2")(grTransFeatures)
grTransFeatures = baseModel.get_layer("batch_normalization_14")(grTransFeatures)
grTransPred = baseModel.get_layer("gr_trans_fc3")(grTransFeatures)
grTransModel = Model(grFeaturesInput, grTransPred)

combinedFeaturesInput = Input(shape=(384,), name='aeLocTimeFeaturesProxy')
combinedTransFeatures = baseModel.get_layer("ae_loc_time_trans_fc1")(combinedFeaturesInput)
combinedTransFeatures = baseModel.get_layer("batch_normalization_15")(combinedTransFeatures)
combinedTransFeatures = baseModel.get_layer("ae_loc_time_trans_fc2")(combinedTransFeatures)
combinedTransFeatures = baseModel.get_layer("batch_normalization_16")(combinedTransFeatures)
combinedTransPred = baseModel.get_layer("ae_loc_time_trans_fc3")(combinedTransFeatures)
combinedTransModel = Model(combinedFeaturesInput, combinedTransPred)

--------------
--------------
--------------
--------------


### Predicting the transient attributes for all hours and months

The `skipCount` variable allows us to control which image would be selected (without needing to adapt much of the dataLoader)

In [6]:
dl = DataLoader("test", 
                includeLocation = True, 
                includeSatellite = True, 
                outputTransientAttributes = True)

skipCount = 10

for batch, _ in dl.loadTestDataInBatches(batchSize):
    if skipCount >= 1:
        skipCount-=1
        continue
    
    grImg, aeImg, locInfo, timeInfo = [batch[i][0:1] for i in range(len(batch))]
    
    dLoc = deprocess_loc(locInfo[0])
    timeInfo = deprocess_time(timeInfo[0])
    print(timeInfo, dLoc)

    
    grFeatures = groundBranchModel.predict_on_batch(grImg)
    aeFeatures = aerialBranchModel.predict_on_batch(aeImg)
    locFeatures = locBranchModel.predict_on_batch(locInfo)
    dLoc = deprocess_loc(locInfo[0])
    
    grTransPred = grTransModel.predict_on_batch(grFeatures)[0]

    timeList, predList, transList = [], [], []
    combinedTransPredList = []
    for month, hour in product(range(1,13), range(24)):
        timeFeatures = timeBranchModel.predict_on_batch(preprocess_time((month, hour)).reshape(1,-1))
        
        concatFV = np.hstack((grFeatures, aeFeatures, locFeatures, timeFeatures))
        pred = consistModel.predict_on_batch(concatFV)
        consistentProb = pred[0][0]
        
        aeLocTimeFV = np.hstack((aeFeatures, locFeatures, timeFeatures))
        combinedTransPred = combinedTransModel.predict_on_batch(aeLocTimeFV)[0]
        combinedTransPredList += [combinedTransPred]
        
        if month == timeInfo[0] and hour == timeInfo[1]:
            gt_aeLocTimeTrans = combinedTransPred
        
        timeList += [(month, hour)]
        predList += [consistentProb]
        transList += [combinedTransPred]
    break
    

sortedTimeList = [(y,x, z) for y,x,z in sorted(zip(predList,timeList, transList), 
                                              key=lambda pair: pair[0], reverse=True)]
sortedProbs = [l[0] for l in sortedTimeList]
sortedTimes = [l[1] for l in sortedTimeList]
sortedTrans = [l[2] for l in sortedTimeList]

((1, 224, 224, 3), (1, 224, 224, 3), (1, 3), (1, 2))
((8, 18), (50.1065184167, 8.678006826950002, 1.862645149230957e-09))
('Correct time: ', (8, 18))
('Top 1% times: ', [])
('Top 5% times: ', [])
('Top 5% pred: ', [], '\n')


### Comparing TAs for a tampered timestamp

In [8]:
from sklearn.metrics import mean_absolute_error as mae


tamperedTimestamp = (12,18)


tamperedTransAtt = [sortedTrans[idx] for idx in range(len(sortedTimes)) if sortedTimes[idx] == tamperedTimestamp][0]
originalTransAtt = [sortedTrans[idx] for idx in range(len(sortedTimes)) if sortedTimes[idx] == timeInfo][0]



print("\nTop 10 differences between aG and aS:")
transDiff = grTransPred - tamperedTransAtt
top10Idx = abs(transDiff).argsort()[-10:][::-1]
for idx in top10Idx:
    print(transLabels[idx], " : ", transDiff[idx]) 

print("\nTop 10 Transient Attributes obtained from GL Image:")
print(timeInfo)
for idx in top10Idx:
    print(transLabels[idx], " : ", grTransPred[idx])
    
print("\nTop 10 Transient Attributes obtained from original timestamp with (OH, Loc, Time):")
print(timeInfo)
for idx in top10Idx:
    print(transLabels[idx], " : ", originalTransAtt[idx])

print("\nTop 10 Transient Attributes obtained from tampered timestamp with (OH, Loc, Time):")
print(tamperedTimestamp)
for idx in top10Idx:
    print(transLabels[idx], " : ", tamperedTransAtt[idx])


Top 10 differences between aG and aS:
('night', ' : ', -0.7273265)
('daylight', ' : ', 0.58510524)
('dark', ' : ', -0.53351223)
('glowing', ' : ', -0.5006038)
('lush', ' : ', 0.39726347)
('sunny', ' : ', 0.25446105)
('spring', ' : ', 0.2500388)
('beautiful', ' : ', -0.22393042)
('summer', ' : ', 0.215341)
('exciting', ' : ', -0.20316222)

Top 10 Transient Attributes obtained from GL Image:
(8, 18)
('night', ' : ', 0.04308346)
('daylight', ' : ', 0.841983)
('dark', ' : ', 0.05488947)
('glowing', ' : ', 0.22586319)
('lush', ' : ', 0.6145125)
('sunny', ' : ', 0.35446912)
('spring', ' : ', 0.3713855)
('beautiful', ' : ', 0.32403362)
('summer', ' : ', 0.4690554)
('exciting', ' : ', 0.25236464)

Top 10 Transient Attributes obtained from original timestamp with (OH, Loc, Time):
(8, 18)
('night', ' : ', 0.15143701)
('daylight', ' : ', 0.758909)
('dark', ' : ', 0.15255195)
('glowing', ' : ', 0.34811044)
('lush', ' : ', 0.5189508)
('sunny', ' : ', 0.3008411)
('spring', ' : ', 0.2826838)
('beaut